### A Big Warning
    WARNING:tensorflow:From <ipython-input-1-32a93b20bd45>:25: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
    WARNING:tensorflow:From C:\Users\VIP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please write your own downloading logic.
    WARNING:tensorflow:From C:\Users\VIP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please use urllib or similar directly.
    Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
    WARNING:tensorflow:From C:\Users\VIP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please use tf.data to implement this functionality.
    Extracting /tmp/data\train-images-idx3-ubyte.gz
    Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
    WARNING:tensorflow:From C:\Users\VIP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please use tf.data to implement this functionality.
    Extracting /tmp/data\train-labels-idx1-ubyte.gz
    WARNING:tensorflow:From C:\Users\VIP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please use tf.one_hot on tensors.
    Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
    Extracting /tmp/data\t10k-images-idx3-ubyte.gz
    Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
    Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
    WARNING:tensorflow:From C:\Users\VIP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
    Instructions for updating:
    Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

In [12]:
import tensorflow as tf

'''
input > weights > hidden layer 1 (activation function) > weights > hidden layer 2
(activation functions) > ... > (weights) > output layer

comapre output to intended output > cost/loss function (cross entropy)
optimzation function (optimizer) > minimize cost (AdamOptimizer....SGD, AdaGrad)

backpropogation

feed forward + backprop = epoch
'''

from tensorflow.examples.tutorials.mnist import input_data

'''
mnist dataset is a collection of 60,000 images of handwritten numbers
and 10,000 images of test dataset

all images of - 28px*28px
28*28 = 784
'''

mnist = input_data.read_data_sets("/tmp/data", one_hot = True)
'''
before one_hot
0 = 0
1 = 1
2 = 2
3 = 3
4 = 4

after one_hot
0 = [1,0,0,0,0]
0 = [0,1,0,0,0]
0 = [0,0,1,0,0]
0 = [0,0,0,1,0]
0 = [0,0,0,0,1]
'''

n_nodes_hidden_layer_1 = 500
n_nodes_hidden_layer_2 = 500
n_nodes_hidden_layer_3 = 500

n_classes = 10
'''
feed and manipulate bathces of 100 images at a time
'''
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    '''
        (input_data*weights) + biases
        Suppose we have all inputs = 0, so no neuron will ever fire.
        Thus, we have biases to add some value and atleast some of the neurons will fire.
    '''
    '''
    These are all the variables for our models
    '''
    hidden_layer_1 = {
        'weights': tf.Variable(tf.random_normal([784, n_nodes_hidden_layer_1])),
        'biases': tf.Variable(tf.random_normal([n_nodes_hidden_layer_1]))
    }
    
    hidden_layer_2 = {
        'weights': tf.Variable(tf.random_normal([n_nodes_hidden_layer_1, n_nodes_hidden_layer_2])),
        'biases': tf.Variable(tf.random_normal([n_nodes_hidden_layer_2]))
    }
    
    hidden_layer_3 = {
        'weights': tf.Variable(tf.random_normal([n_nodes_hidden_layer_2, n_nodes_hidden_layer_3])),
        'biases': tf.Variable(tf.random_normal([n_nodes_hidden_layer_3]))
    }
    
    output_layer = {
        'weights': tf.Variable(tf.random_normal([n_nodes_hidden_layer_3, n_classes])),
        'biases': tf.Variable(tf.random_normal([n_classes]))
    }
    
    # (input_data * weights) + biases
    '''
    This is our model
    '''
    layer_1 = tf.add(tf.matmul(data, hidden_layer_1['weights']), hidden_layer_1['biases'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, hidden_layer_2['weights']), hidden_layer_2['biases'])
    layer_2 = tf.nn.relu(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2, hidden_layer_3['weights']), hidden_layer_3['biases'])
    layer_3 = tf.nn.relu(layer_3)
    
    output = tf.add(tf.matmul(layer_3, output_layer['weights']), output_layer['biases'])
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
    
    # default learning_rate = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #cycles feed_forward + backprop
    total_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(total_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                __, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            
            print('Epoch: ', epoch, ' completed out of ', total_epochs, '; loss: ', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy: ', accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
        
train_neural_network(x)

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch:  0  completed out of  10 ; loss:  1890097.91806
Epoch:  1  completed out of  10 ; loss:  413755.420845
Epoch:  2  completed out of  10 ; loss:  221301.929931
Epoch:  3  completed out of  10 ; loss:  133515.919708
Epoch:  4  completed out of  10 ; loss:  81034.8499086
Epoch:  5  completed out of  10 ; loss:  51808.9523071
Epoch:  6  completed out of  10 ; loss:  36297.1939351
Epoch:  7  completed out of  10 ; loss:  26585.0916518
Epoch:  8  completed out of  10 ; loss:  21116.0054017
Epoch:  9  completed out of  10 ; loss:  19245.8315371
Accuracy:  0.9542


The better the model the more the accuracy.